In [3]:
!pip install torchsummary

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
import pickle
from PIL import Image
import torchsummary
import warnings
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



## Load the data

In [5]:
class TestData(torch.utils.data.Dataset):
    def __init__(self, file_path, transform=None):
        self.data = None
        with open(file_path, "rb") as f:
            self.data = pickle.load(f)
            self.images = self.data[b"data"]
            self.ids = self.data[b"ids"]
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        img = self.images[index]
        id_ = self.ids[index]

        # Convert image to PIL Image
        img = img.reshape(3, 32, 32).transpose(1, 2, 0)
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return id_, img


In [6]:
def augment_data(input_dim=(3, 32, 32)):
    transform_train = transforms.Compose(
        [
            # transforms.RandomVerticalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(
                brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1
            ),
            transforms.RandomCrop(input_dim[1], padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )

    transform_val_test = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )
    return transform_train, transform_val_test

In [7]:
# def load_data(input_dim=(3, 32, 32)):
#     import os

#     data_directory = os.path.dirname(__file__) + "/../data"
#     test_path = os.path.join(data_directory, "testdata", "cifar_test_nolabels.pkl")
data_directory = "./data"
input_dim = (3,32,32)
test_path = "/kaggle/input/test-set/cifar_test_nolabels.pkl"

transform_train, transform_val_test = augment_data(input_dim)

trainset = datasets.CIFAR10(
    root=data_directory, train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

val_set = datasets.CIFAR10(
    root=data_directory, train=False, download=True, transform=transform_val_test
)
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=100, shuffle=False, num_workers=2
)

test_set = TestData(file_path=test_path, transform=transform_val_test)
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=1, shuffle=False, num_workers=2
)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

#     return trainloader, val_loader, testloader, classes


100%|██████████| 170498071/170498071 [00:03<00:00, 49781367.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# for batch_idx, samples in enumerate(val_loader):
#       print(batch_idx, samples)

# define model

In [9]:
class ResidualBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(self.expansion * planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [10]:
class MiniResNet(nn.Module):
    def __init__(self, num_blocks=(2, 2, 2, 2)):
        super(MiniResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, 10)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [11]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float("inf")

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


def get_optimizers(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    early_stopper = EarlyStopper(patience=10, min_delta=10)

    return criterion, optimizer, scheduler, early_stopper

In [14]:
model = MiniResNet(num_blocks=[1, 1, 1, 1]).cuda()

In [15]:
torchsummary.summary(model, input_dim)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
     ResidualBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8          [-1, 128, 16, 16]          73,728
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,456
      BatchNorm2d-11          [-1, 128, 16, 16]             256
           Conv2d-12          [-1, 128, 16, 16]           8,192
      BatchNorm2d-13          [-1, 128, 16, 16]             256
    ResidualBlock-14          [-1, 128,

In [16]:
model

MiniResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchN

# Train the model

In [18]:
import torch.backends.cudnn as cudnn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
device

'cuda'

In [19]:
criterion, optimizer, scheduler, early_stopper = get_optimizers(model)

In [20]:
epochs = 30

In [21]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for _, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return train_loss, correct, total


In [22]:
def train(
    model,
    train_loader,
    test_loader,
    epochs,
    criterion,
    optimizer,
    scheduler,
    early_stopper,
    device,
):
    train_loss_history = []
    train_acc_history = []
    test_loss_history = []
    test_acc_history = []

    for epoch in range(epochs):
        train_loss, train_correct, train_total = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        test_loss, test_correct, test_total = test(
            model, test_loader, criterion, device
        )

        train_loss = train_loss / len(train_loader)
        test_loss = test_loss / len(test_loader)

        train_acc = train_correct / train_total
        test_acc = test_correct / test_total

        train_loss_history += [train_loss]
        test_loss_history += [test_loss]

        train_acc_history.append(train_acc)
        test_acc_history.append(test_acc)

        print(
            f"Epoch {epoch + 1}, Train loss {train_loss:.3f}, Test loss {test_loss:.3f}, Train Accuracy: {train_acc:.3f}, Test Accuracy: {test_acc:.3f}"
        )
        scheduler.step()

        if (epoch % 10 == 0) or early_stopper.early_stop(test_loss):
            state = {
                "epoch": epoch,
                "state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": test_loss,
            }
            if not os.path.isdir("checkpoint"):
                os.mkdir("checkpoint")
                torch.save(state, "./checkpoint/ckpt.pth")

In [23]:
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for _, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss, correct, total

In [29]:
 train(
        model,
        trainloader,
        val_loader,
        epochs,
        criterion,
        optimizer,
        scheduler,
        early_stopper,
        device,
    )

Epoch 1, Train loss 0.657, Test loss 0.897, Train Accuracy: 0.771, Test Accuracy: 0.709
Epoch 2, Train loss 0.596, Test loss 0.609, Train Accuracy: 0.791, Test Accuracy: 0.796
Epoch 3, Train loss 0.541, Test loss 0.617, Train Accuracy: 0.812, Test Accuracy: 0.799
Epoch 4, Train loss 0.501, Test loss 0.735, Train Accuracy: 0.826, Test Accuracy: 0.770
Epoch 5, Train loss 0.472, Test loss 0.559, Train Accuracy: 0.836, Test Accuracy: 0.812
Epoch 6, Train loss 0.443, Test loss 0.568, Train Accuracy: 0.846, Test Accuracy: 0.814
Epoch 7, Train loss 0.416, Test loss 0.601, Train Accuracy: 0.857, Test Accuracy: 0.816
Epoch 8, Train loss 0.396, Test loss 0.528, Train Accuracy: 0.863, Test Accuracy: 0.830
Epoch 9, Train loss 0.371, Test loss 0.626, Train Accuracy: 0.873, Test Accuracy: 0.795
Epoch 10, Train loss 0.353, Test loss 0.624, Train Accuracy: 0.877, Test Accuracy: 0.809
Epoch 11, Train loss 0.340, Test loss 0.406, Train Accuracy: 0.882, Test Accuracy: 0.863
Epoch 12, Train loss 0.325, Te

In [34]:
    valid_loss, valid_correct, valid_total = test(model, val_loader, criterion, device)

    valid_acc = valid_correct / valid_total
    print(f"Valid Accuracy: {valid_acc}")

#     test_loss, test_correct, test_total = test(model, test_loader, criterion, DEVICE)

#     test_acc = test_correct / test_total
#     print(f"Test Accuracy: {test_acc}")

#     results = infer(model, test_loader.dataset, criterion, DEVICE)
#     print(results)

Valid Accuracy: 0.8839


0.8839

In [36]:
def infer(model, test_loader, criterion, device):
    print(len(test_loader))
    model.eval()

    results = []

    for _, (id_, image) in enumerate(test_loader):
        image = image.to(device)
        output = model(image)
        _, predicted = output.max(1)
        results.append({"ID": id_.item(), "Labels": predicted.item()})

    return pd.DataFrame(results)

In [40]:
results = infer(model, testloader, criterion, device)


10000


In [47]:
results.head(80)

,ID,Labels
0,0,8
1,1,8
2,2,8
3,3,8
4,4,8
...,...,...
75,75,8
76,76,8
77,77,4
78,78,8


In [52]:
np.unique(results['Labels'],return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 997,  890,  661, 1277, 1378, 1063,  864,  804,  967, 1099]))

In [ ]:
results.to_csv('/kaggle/working/results.csv', index =False)

# Number of parameters for block combination

In [64]:
model_2= MiniResNet(num_blocks=[1, 1,1, 1]).cuda()
torchsummary.summary(model_2, input_dim)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
     ResidualBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8          [-1, 128, 16, 16]          73,728
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,456
      BatchNorm2d-11          [-1, 128, 16, 16]             256
           Conv2d-12          [-1, 128, 16, 16]           8,192
      BatchNorm2d-13          [-1, 128, 16, 16]             256
    ResidualBlock-14          [-1, 128,